# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [3]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [5]:
# cleaning & feature engineering
spaceship = spaceship.dropna()
spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x[0])
spaceship = spaceship.drop(columns=['PassengerId', 'Name'])
spaceship = pd.get_dummies(spaceship, columns=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP'])

In [6]:
# define features and target
spaceship_features = spaceship.drop(columns='Transported')
spaceship_target = spaceship['Transported']

# split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(spaceship_features, spaceship_target, test_size=0.2, random_state=0)

# standardize the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gb_reg = GradientBoostingClassifier(max_depth=8, n_estimators=100)
gb_reg.fit(X_train_scaled, y_train)

GradientBoostingClassifier(max_depth=8)

- Evaluate your model

In [8]:
print("Gradient Boosting accuracy", gb_reg.score(X_test_scaled, y_test))

Gradient Boosting accuracy 0.783661119515885


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [9]:
# define hyperparameter grid
hyper_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [4, 6, 8, 10, 12]
}

- Run Grid Search

In [11]:
# perform grid search
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(gb_reg, hyper_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(max_depth=8),
             param_grid={'max_depth': [4, 6, 8, 10, 12],
                         'n_estimators': [50, 100, 150, 200]},
             scoring='accuracy')

- Evaluate your model

In [12]:
# evaluate the best model
best_gb = grid_search.best_estimator_
print("Best Gradient Boosting accuracy", best_gb.score(X_test_scaled, y_test))
print("Best Gradient Boosting hyperparameters", grid_search.best_params_)
print("Best Gradient Boosting feature importances", best_gb.feature_importances_)

Best Gradient Boosting accuracy 0.791981845688351
Best Gradient Boosting hyperparameters {'max_depth': 4, 'n_estimators': 100}
Best Gradient Boosting feature importances [5.33404352e-02 7.65198564e-02 8.44517064e-02 5.89943872e-02
 1.09206683e-01 1.18372135e-01 5.77659827e-02 1.13385944e-02
 7.68251562e-03 2.53681323e-01 1.12077610e-01 1.32949542e-03
 3.85907411e-04 2.85216003e-03 3.76778668e-04 2.27674876e-02
 2.61555869e-03 1.18676722e-02 0.00000000e+00 4.35585543e-03
 1.24526407e-03 8.30748068e-03 4.43237662e-04 2.18730231e-05]
